In [1]:
import os 
os.environ["TFF_CPP_MIN_LOG_LEVEL"]="2"

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

In [8]:
# #for gpu
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [10]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


## Sequential Api


In [17]:
s_model = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, 3, padding="valid", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)


**Note**
- padding= 'valid' means the spatial shape will change according to  kernal shape. and 'same' will keep unchange the shape.
- layers.Conv2D(32,3): first argument 32 is the number of kernals/filters and the 3 represents the filter/kernal size.

In [20]:
#model configuration
s_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"],
)

In [21]:
#fit model
s_model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2)

Epoch 1/10
782/782 - 43s - loss: 1.3722 - accuracy: 0.5036 - 43s/epoch - 56ms/step
Epoch 2/10
782/782 - 42s - loss: 1.1388 - accuracy: 0.5987 - 42s/epoch - 54ms/step
Epoch 3/10
782/782 - 37s - loss: 0.9975 - accuracy: 0.6522 - 37s/epoch - 48ms/step
Epoch 4/10
782/782 - 35s - loss: 0.8880 - accuracy: 0.6902 - 35s/epoch - 45ms/step
Epoch 5/10
782/782 - 35s - loss: 0.8184 - accuracy: 0.7139 - 35s/epoch - 44ms/step
Epoch 6/10
782/782 - 35s - loss: 0.7497 - accuracy: 0.7412 - 35s/epoch - 45ms/step
Epoch 7/10
782/782 - 42s - loss: 0.6973 - accuracy: 0.7557 - 42s/epoch - 54ms/step
Epoch 8/10
782/782 - 44s - loss: 0.6509 - accuracy: 0.7729 - 44s/epoch - 56ms/step
Epoch 9/10
782/782 - 42s - loss: 0.6072 - accuracy: 0.7862 - 42s/epoch - 53ms/step
Epoch 10/10
782/782 - 35s - loss: 0.5743 - accuracy: 0.7995 - 35s/epoch - 45ms/step


In [22]:
#model evaluate
s_model.evaluate(x_test,y_test,batch_size=32,verbose=2)

313/313 - 2s - loss: 0.8092 - accuracy: 0.7341 - 2s/epoch - 7ms/step


[0.8091689944267273, 0.7340999841690063]

In [23]:
print(s_model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 flatten_3 (Flatten)         (None, 2048)             

## functional API

In [25]:
## functional model

inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, 3)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(10)(x)
f_model = keras.Model(inputs=inputs, outputs=outputs)

**Note**
- if we use batch normalization then after normalization we use activation function
- MaxPooling2D(): empty argument represents pool_size as 2,2

In [26]:
# model configuration
f_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"]
)

In [27]:
# model fit
f_model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2)

Epoch 1/10
782/782 - 70s - loss: 1.2461 - accuracy: 0.5571 - 70s/epoch - 89ms/step
Epoch 2/10
782/782 - 60s - loss: 0.8879 - accuracy: 0.6885 - 60s/epoch - 77ms/step
Epoch 3/10
782/782 - 55s - loss: 0.7440 - accuracy: 0.7409 - 55s/epoch - 71ms/step
Epoch 4/10
782/782 - 55s - loss: 0.6486 - accuracy: 0.7727 - 55s/epoch - 71ms/step
Epoch 5/10
782/782 - 55s - loss: 0.5636 - accuracy: 0.8017 - 55s/epoch - 71ms/step
Epoch 6/10
782/782 - 56s - loss: 0.4970 - accuracy: 0.8261 - 56s/epoch - 71ms/step
Epoch 7/10
782/782 - 56s - loss: 0.4362 - accuracy: 0.8491 - 56s/epoch - 71ms/step
Epoch 8/10
782/782 - 55s - loss: 0.3736 - accuracy: 0.8698 - 55s/epoch - 70ms/step
Epoch 9/10
782/782 - 55s - loss: 0.3295 - accuracy: 0.8836 - 55s/epoch - 70ms/step
Epoch 10/10
782/782 - 55s - loss: 0.2887 - accuracy: 0.8974 - 55s/epoch - 70ms/step


In [28]:
#evaluate model
f_model.evaluate(x_test,y_test,batch_size=64,verbose=2)

157/157 - 3s - loss: 1.2035 - accuracy: 0.6878 - 3s/epoch - 19ms/step


[1.2034752368927002, 0.6877999901771545]

**Note** in the functional model the train accuracy is about 90 whereas the test aacuracy is 68. Thererfore here overfitting happens. We will work on this later. to overcome overfitting we will use regularization in the next section./

### SUGGESTION
- try tweaking the kernal size, training time e.t.c hyperparameters
